#### 1. Imports and Data Loading

In [327]:
import pandas as pd
import ast

# Paths
purchases_file = "../data/INPUT/holded_purchases/clean/holded_purchases_clean.csv"
contacts_file = "../data/INPUT/holded_contacts/clean/holded_contacts_clean.csv"
metrics_summary_file = "../data/OUTPUT/2025-07/metrics_summary.csv"  # adjust folder if needed

# Load data
purchases = pd.read_csv(purchases_file)
contacts = pd.read_csv(contacts_file)
df_metrics = pd.read_csv(metrics_summary_file)

print("Data shapes:")
print("Purchases:", purchases.shape)
print("Contacts:", contacts.shape)
print("Metrics:", df_metrics.shape)




Data shapes:
Purchases: (433, 29)
Contacts: (500, 27)
Metrics: (17, 20)


In [332]:
contacts.columns

Index(['id', 'customId', 'name', 'code', 'vatnumber', 'tradeName', 'email',
       'mobile', 'phone', 'type', 'iban', 'swift', 'groupId', 'clientRecord',
       'supplierRecord', 'billAddress', 'customFields', 'defaults',
       'socialNetworks', 'tags', 'notes', 'contactPersons',
       'shippingAddresses', 'isperson', 'createdAt', 'updatedAt',
       'updatedHash'],
      dtype='object')

#### 2. Supplier Mapping Table

In [328]:
# Keep only suppliers and their tags
supplier_tags_map = contacts[contacts['type'] == 'supplier'][['id', 'tags']].copy()

def parse_tags(val):
    """Converts raw tag strings (like '["opex", "tool"]') into a list."""
    if isinstance(val, list):
        return [t.strip().lower().replace('"', '') for t in val]
    if pd.isna(val) or val == '':
        return []
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, list):
                return [t.strip().lower().replace('"', '') for t in parsed]
        except (ValueError, SyntaxError):
            return [val.strip().lower().replace('"', '')]
    return []

# Parse tags in contacts
supplier_tags_map['tags'] = supplier_tags_map['tags'].apply(parse_tags)
print("Sample supplier tags:")
print(supplier_tags_map.head(10))


Sample supplier tags:
                         id                    tags
0  6694f45fc6f6db032502c35b      [opex, prod, tool]
1  6694f5a3919746067d00ab14   [opex, ops, staffing]
2  6694f65bad0998298507286e     [opex, fin, vendor]
3  6694f697d25266d2f009e339     [opex, fin, vendor]
4  6694f7811cbbbbcbc5094183  [opex, prod, staffing]
5  6694f881e793fd466106a51f      [cogs, tech, tool]
6  6694f924d7d911379105f312       [opex, ops, tool]
7  6694faff0c3b83b17508c837   [cac, sale, staffing]
8  6694fc239e07ff6dfc052826       [cac, sale, tool]
9  6694febb64ba6319900bcdcf      [cogs, tech, tool]


#### 3. Merge Purchases with Tags

In [329]:
# Merge purchases with supplier tags
purchases_with_tags = purchases.merge(
    supplier_tags_map, left_on='contact', right_on='id', how='left'
)

# Rename 'tags' column (from contacts) to a standard name
if 'tags_y' in purchases_with_tags.columns:
    purchases_with_tags.rename(columns={'tags_y': 'tags'}, inplace=True)
elif 'tags' not in purchases_with_tags.columns:
    purchases_with_tags['tags'] = [[] for _ in range(len(purchases_with_tags))]

# Convert purchase date to datetime and extract month
purchases_with_tags['date'] = pd.to_datetime(purchases_with_tags['date'], errors='coerce')
purchases_with_tags['month'] = purchases_with_tags['date'].dt.to_period('M').astype(str)

print("Purchases with tags preview:")
print(purchases_with_tags[['contact', 'tags', 'month']].head(10))


Purchases with tags preview:
                    contact                tags    month
0  6694f881e793fd466106a51f  [cogs, tech, tool]  2025-07
1  67ae52d039bbf51a6108472b  [cogs, tech, tool]  2025-07
2  681a5579f1ea6b91650b2f65  [cogs, tech, tool]  2025-07
3  6694f881e793fd466106a51f  [cogs, tech, tool]  2025-07
4  681a5579f1ea6b91650b2f65  [cogs, tech, tool]  2025-07
5  672ca6594cc0add3080b6460  [cogs, tech, tool]  2025-07
6  66bb168bffa1a257fd08a465   [opex, mkg, tool]  2025-07
7  67be15a371730ffc8e081860  [cogs, tech, tool]  2025-07
8  6694febb64ba6319900bcdcf  [cogs, tech, tool]  2025-07
9  686f9d2da3972b9b98032aff                 NaN  2025-07


#### 5. Calculate OPEX, COGS, Financial Costs

In [330]:
# Ensure 'tags' column is always a list
purchases_with_tags['tags'] = purchases_with_tags['tags'].apply(
    lambda x: x if isinstance(x, list) else []
)

# Filter by tags
opex_data = purchases_with_tags[purchases_with_tags['tags'].apply(lambda t: 'opex' in t)]
cogs_data = purchases_with_tags[purchases_with_tags['tags'].apply(lambda t: 'cogs' in t)]
fin_data = purchases_with_tags[purchases_with_tags['tags'].apply(lambda t: 'costes financieros' in t)]

# Summaries by month
opex_by_month = opex_data.groupby('month')['total'].sum().reset_index().rename(columns={'total': 'opex_calc'})
cogs_by_month = cogs_data.groupby('month')['total'].sum().reset_index().rename(columns={'total': 'cogs_calc'})
fin_by_month = fin_data.groupby('month')['total'].sum().reset_index().rename(columns={'total': 'fin_calc'})

# Print results
print("Rows with 'opex':", len(opex_data))
print("OPEX by month:\n", opex_by_month)
print("\nRows with 'cogs':", len(cogs_data))
print("COGS by month:\n", cogs_by_month)
print("\nRows with 'costes financieros':", len(fin_data))
print("Financial Costs by month:\n", fin_by_month)

Rows with 'opex': 222
OPEX by month:
       month  opex_calc
0   2024-03       4.62
1   2024-04    7689.21
2   2024-05   13122.96
3   2024-06   10834.95
4   2024-07   19037.16
5   2024-08    9515.14
6   2024-09   24717.66
7   2024-10   12650.11
8   2024-11   12852.10
9   2024-12   13760.72
10  2025-01   13816.92
11  2025-02   20243.17
12  2025-03   12671.73
13  2025-04   20386.01
14  2025-05   13356.06
15  2025-06   13221.77
16  2025-07      36.00

Rows with 'cogs': 134
COGS by month:
       month  cogs_calc
0   2024-06     989.79
1   2024-07      96.89
2   2024-08     492.90
3   2024-09     560.42
4   2024-10    1041.59
5   2024-11    7154.85
6   2024-12     513.45
7   2025-01    4580.42
8   2025-02    9463.69
9   2025-03    1311.20
10  2025-04    3186.33
11  2025-05    3746.13
12  2025-06    5033.46
13  2025-07    4286.83

Rows with 'costes financieros': 3
Financial Costs by month:
      month  fin_calc
0  2025-02    729.00
1  2025-03   1271.96
2  2025-06   2462.72


#### 5. Merge with Metrics Summary

In [331]:
# Ensure 'tags' column is always a list
purchases_with_tags['tags'] = purchases_with_tags['tags'].apply(
    lambda x: x if isinstance(x, list) else []
)

# Filter by tags
opex_data = purchases_with_tags[purchases_with_tags['tags'].apply(lambda t: 'opex' in t)]
cogs_data = purchases_with_tags[purchases_with_tags['tags'].apply(lambda t: 'cogs' in t)]
fin_data = purchases_with_tags[purchases_with_tags['tags'].apply(lambda t: 'costes financieros' in t)]

# Summaries by month
opex_by_month = opex_data.groupby('month')['total'].sum().reset_index().rename(columns={'total': 'opex_calc'})
cogs_by_month = cogs_data.groupby('month')['total'].sum().reset_index().rename(columns={'total': 'cogs_calc'})
fin_by_month = fin_data.groupby('month')['total'].sum().reset_index().rename(columns={'total': 'fin_calc'})

# Print results
print("Rows with 'opex':", len(opex_data))
print("OPEX by month:\n", opex_by_month)
print("\nRows with 'cogs':", len(cogs_data))
print("COGS by month:\n", cogs_by_month)
print("\nRows with 'costes financieros':", len(fin_data))
print("Financial Costs by month:\n", fin_by_month)


Rows with 'opex': 222
OPEX by month:
       month  opex_calc
0   2024-03       4.62
1   2024-04    7689.21
2   2024-05   13122.96
3   2024-06   10834.95
4   2024-07   19037.16
5   2024-08    9515.14
6   2024-09   24717.66
7   2024-10   12650.11
8   2024-11   12852.10
9   2024-12   13760.72
10  2025-01   13816.92
11  2025-02   20243.17
12  2025-03   12671.73
13  2025-04   20386.01
14  2025-05   13356.06
15  2025-06   13221.77
16  2025-07      36.00

Rows with 'cogs': 134
COGS by month:
       month  cogs_calc
0   2024-06     989.79
1   2024-07      96.89
2   2024-08     492.90
3   2024-09     560.42
4   2024-10    1041.59
5   2024-11    7154.85
6   2024-12     513.45
7   2025-01    4580.42
8   2025-02    9463.69
9   2025-03    1311.20
10  2025-04    3186.33
11  2025-05    3746.13
12  2025-06    5033.46
13  2025-07    4286.83

Rows with 'costes financieros': 3
Financial Costs by month:
      month  fin_calc
0  2025-02    729.00
1  2025-03   1271.96
2  2025-06   2462.72
